In [5]:
%pip install tqdm pandas
%pip install --extra-index-url https://test.pypi.org/simple/ git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-OOEP
# NOTE local repo
#%pip install --extra-index-url https://test.pypi.org/simple/ ../../..
%pip install git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://test.pypi.org/simple/
  Cloning https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-OOEP to c:\users\asus pro\appdata\local\temp\pip-req-build-d2t7ufba
  Resolved https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-OOEP to commit f1f441d093e394e6ce022872a0e0ccb375d84d13
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-OOEP 'C:\Users\Asus Pro\AppData\Local\Temp\pip-req-build-d2t7ufba'


  Cloning https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git to c:\users\asus pro\appdata\local\temp\pip-req-build-8mokt1rt
  Resolved https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git to commit 62b02ce49492fb063dd0181b078ab37cb7202d4e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git 'C:\Users\Asus Pro\AppData\Local\Temp\pip-req-build-8mokt1rt'
  Running command git submodule update --init --recursive -q


In [16]:
import energyplus.ooep as ooep
import energyplus.ooep.ems
import energyplus.ooep.addons.state

from energyplus.dataset.basic import dataset as epds

env = energyplus.ooep.ems.Environment().__enter__()

keys = ['CHILLER', 'CHILLER 1', 'CHILLER 2']
emeter = [env.variable(dict(variable_name='Chiller Electricity Rate', variable_key=key)) for key in keys]

T_outdoor = env.variable(dict(
    variable_name='Site Outdoor Air Drybulb Temperature',
    variable_key='ENVIRONMENT'
))

F_chw = env.variable(dict(variable_name='System Node Mass Flow Rate', variable_key='CHW Loop Supply Side'))
T_chws = env.variable(dict(variable_name='Chiller Evaporator Outlet Temperature', variable_key='Chiller'))
T_chwr = env.variable(dict(variable_name='Chiller Evaporator Inlet Temperature', variable_key='Chiller'))

T_chws_set = env.actuator(dict(component_type='System Node Setpoint',control_type = 'Temperature Setpoint', actuator_key='CHW Loop Supply Side Outlet'))


sm_env = ooep.addons.state.StateMachine(env)
sm_stepf = sm_env.step_function(
    dict(event_name='begin_zone_timestep_after_init_heat_balance')
)

sm_env.run(
    '--output-directory', 'build3/demo-eplus',
    '--weather', f'./USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
    f'./test9_timeStepReduced.idf',
    verbose=True,
)
print('output', 'build3/demo-eplus',  file=__import__('sys').stderr)




output build3/demo-eplus


In [18]:
import numpy as np

# Define Q-learning parameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate

# Define state and action spaces
num_actions = 3  # Increase, remain same, decrease
num_states = 100  # Number of discrete states

# Initialize Q-table
Q = np.zeros((num_states, num_actions))

# Define variables to store training metrics
total_rewards = []

# Define state discretization parameters
min_cooling_load = 0
max_cooling_load = 1000
min_outdoor_temp = -10
max_outdoor_temp = 40

def calculate_cooling_load(T_chwr, T_chws, C_p=4200, rho=1000):
    return (C_p * rho * (T_chwr - T_chws)) / 3600

def calculate_reward(T_chwr, x_comfort, x_energy,  P_chiller, P_chiller_nominal=60000, beta1=0.25, beta2=4.15, beta3=-1, beta4=1):
    U_comfort = 1 / (1 + beta1 * np.exp(beta2 * (T_chwr - 6)))
    U_energy =  beta3 * ((np.sum(P_chiller))/P_chiller_nominal) + beta4
    return U_comfort * x_comfort + U_energy * x_energy

# Define function to discretize state
def discretize_state(cooling_load, outdoor_temp):
    cooling_load = np.clip(cooling_load, min_cooling_load, max_cooling_load)
    outdoor_temp = np.clip(outdoor_temp, min_outdoor_temp, max_outdoor_temp)
    cooling_load_scaled = (cooling_load - min_cooling_load) / (max_cooling_load - min_cooling_load)
    outdoor_temp_scaled = (outdoor_temp - min_outdoor_temp) / (max_outdoor_temp - min_outdoor_temp)
    state = int(cooling_load_scaled * (num_states - 1))
    return state

# Define function to select action using epsilon-greedy policy
def select_action(state):
    if np.random.rand() < epsilon:
        action = np.random.choice(num_actions)   # Exploration
        action_mapping = {
        0: -1,  # Decrease setpoint temperature by 1 degree
        1: 0,   # Maintain current setpoint temperature
        2: 1    # Increase setpoint temperature by 1 degree
        }

        # Update Setpoint Temperature based on Chosen Action
        delta_setpoint = action_mapping[action]
        new_setpoint = T_chws_set.value + delta_setpoint
        T_chws_set.value = new_setpoint
        return action  # Exploration
    else:
        action = np.argmax(Q[state])  # Exploitation
        action_mapping = {
        0: -1,  # Decrease setpoint temperature by 1 degree
        1: 0,   # Maintain current setpoint temperature
        2: 1    # Increase setpoint temperature by 1 degree
        }

        # Update Setpoint Temperature based on Chosen Action
        delta_setpoint = action_mapping[action]
        new_setpoint = T_chws_set.value + delta_setpoint
        T_chws_set.value = new_setpoint
        return action

# Define function to update Q-table
def update_Q_table(state, action, reward, next_state):
    best_next_action = np.argmax(Q[next_state])
    Q[state, action] += alpha * (reward + gamma * Q[next_state, best_next_action] - Q[state, action])

In [23]:
while True:
    total_reward = 0 

        
        # Get next state
    sm_stepf()
    next_cooling_load = calculate_cooling_load(T_chwr.value, T_chws.value)
    next_state = discretize_state(next_cooling_load, T_outdoor.value)

        # Update Q-table
    update_Q_table(state, action, reward, next_state)
        
        # Update variables for next iteration
    state = next_state
    total_reward += reward
    total_rewards.append(total_reward)
    
        # Print episode stats
    try:print(rf"emeter={emeter[0].value,emeter[1].value,emeter[2].value}) (T_outdoor={T_outdoor.value}) (F_chw={F_chw.value}) (T_chws={T_chws.value}) (T_chwr={T_chwr.value}) (T_chws_set={T_chws_set.value} Total Reward = {total_reward}", flush=True, file=__import__('sys').stderr)
    except ooep.ems.NotReadyError: pass

    except ooep.ems.NotReadyError: pass
    

# Testing and Evaluation
# Run the trained agent in the simulation environment and collect performance metrics

# Results Analysis
# Analyze the performance of the Q-learning-based controller


SyntaxError: expected 'except' or 'finally' block (2440802703.py, line 20)